<a href="https://colab.research.google.com/github/luisosmx/Python_exercises/blob/main/Cloud_storage_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-cloud-storage
!pip install pandas google-cloud-storage

In [19]:
from google.cloud import storage
import os
import csv
import json


# Configura la variable de entorno para las credenciales
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/projectf1-data-7faef3aab5d9.json'

# Crea un cliente de almacenamiento
client = storage.Client()

bucket_name = 'bucketdata_storage'
blob_name = 'Declaratorias2000_2016.csv'
destination_file_name = 'file.csv'

# Descargar el archivo CSV desde Cloud Storage
bucket = client.get_bucket(bucket_name)
blob = bucket.blob(blob_name)
blob.download_to_filename(destination_file_name)

# Convertir el archivo CSV a JSON
csv_data = []
with open(destination_file_name, 'r', encoding='ISO-8859-1') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        csv_data.append(row)



keys_json = {'Identificador': 'id',
             'Clave Estado': 'Clave_Estado',
             'Clave Municipio': 'Clave_Municipio',
             'Tipo Declaratoria': 'Tipo_Declaratoria',
             'Clasificación Fenómeno': 'Clasificacion_fenomeno',
             'Tipo Fenómeno': 'Tipo_Fenomeno',
             'Fecha Inicio': 'Fecha_Inicio',
             'Fecha Fin': 'Fecha_Fin'
             }

for item in csv_data:
    for old_key, new_key in keys_json.items():
        if old_key in item:
            item[new_key] = item.pop(old_key)

modified_json = json.dumps(csv_data[:10], indent=4)

print(modified_json)

[
    {
        "Estado": "Baja California",
        "Municipio": "Tijuana",
        "Fecha Publicaci\u00f3n": "09/05/2001",
        "Observaciones": "Lluvia Atipica e Impredecible",
        "id": "176",
        "Clave_Estado": "2",
        "Clave_Municipio": "2004",
        "Tipo_Declaratoria": "Desastre",
        "Clasificacion_fenomeno": "Hidrometeorol\u00f3gico",
        "Tipo_Fenomeno": "Lluvias",
        "Fecha_Inicio": "01/01/2001",
        "Fecha_Fin": "28/02/2001"
    },
    {
        "Estado": "M\u00e9xico",
        "Municipio": "Tenango del Valle",
        "Fecha Publicaci\u00f3n": "07/06/2001",
        "Observaciones": "Lluvia Atipica e Impredecible",
        "id": "178",
        "Clave_Estado": "15",
        "Clave_Municipio": "15090",
        "Tipo_Declaratoria": "Desastre",
        "Clasificacion_fenomeno": "Hidrometeorol\u00f3gico",
        "Tipo_Fenomeno": "Lluvias",
        "Fecha_Inicio": "23/04/2001",
        "Fecha_Fin": "23/04/2001"
    },
    {
        "Estado": 

In [ ]:
json_data = json.dumps(csv_data, indent=4)
print(json_data)
# Escribir el resultado en un archivo JSON
with open('data.json', 'w') as json_file:
    json_file.write(json_data)

In [ ]:
max_elements = 2000

for index, item in enumerate(json_data):
    if index >= max_elements:
        break
    formatted_item = str(item).replace(" ", "")
    print(formatted_item)

#data = json.loads(formatted_item)

#print(json.dumps(data, indent=4))

[






{










"
I
d
e
n
t
i
f
i
c
a
d
o
r
"
:

"
1
7
6
"
,










"
E
s
t
a
d
o
"
:

"
B
a
j
a

C
a
l
i
f
o
r
n
i
a
"
,










"
C
l
a
v
e

E
s
t
a
d
o
"
:

"
2
"
,










"
M
u
n
i
c
i
p
i
o
"
:

"
T
i
j
u
a
n
a
"
,










"
C
l
a
v
e

M
u
n
i
c
i
p
i
o
"
:

"
2
0
0
4
"
,










"
T
i
p
o

D
e
c
l
a
r
a
t
o
r
i
a
"
:

"
D
e
s
a
s
t
r
e
"
,










"
C
l
a
s
i
f
i
c
a
c
i
\
u
0
0
f
3
n

F
e
n
\
u
0
0
f
3
m
e
n
o
"
:

"
H
i
d
r
o
m
e
t
e
o
r
o
l
\
u
0
0
f
3
g
i
c
o
"
,










"
T
i
p
o

F
e
n
\
u
0
0
f
3
m
e
n
o
"
:

"
L
l
u
v
i
a
s
"
,










"
F
e
c
h
a

P
u
b
l
i
c
a
c
i
\
u
0
0
f
3
n
"
:

"
0
9
/
0
5
/
2
0
0
1
"
,










"
F
e
c
h
a

I
n
i
c
i
o
"
:

"
0
1
/
0
1
/
2
0
0
1
"
,










"
F
e
c
h
a

F
i
n
"
:

"
2
8
/
0
2
/
2
0
0
1
"
,










"
O
b
s
e
r
v
a
c
i
o
n
e
s
"
:

"
L
l
u
v
i
a

A
t
i
p
i
c
a

e

I
m
p
r
e
d
e
c
i
b
l
e
"






}
,






{










"
I
d
e
n
t
i
f
i
c
a
d
o
r
"
:

"
1
7
8
"
,










"
E
s
t
a
d
o
"
:

"
M
\
u
0
0
e
9
